## Ref

+ Dynamic Matrix Factorization with Priors on Unknown Values
+ https://arxiv.org/pdf/1507.06452.pdf

### imports

In [1]:
import numpy as np
import csv
import random
import time
import copy
import pickle

### Load data

In [2]:
train = []
update = []
test = []

with open("movielens_online_train.csv", newline = "") as f:
    rows = csv.reader(f)
    for row in rows:
        train.append(row)
        
with open("movielens_online_test.csv", newline = "") as f:
    rows = csv.reader(f)
    for row in rows:
        test.append(row)

In [3]:
train[:5]

[['429', '22', '4.0', '828124615'],
 ['429', '150', '5.0', '828124615'],
 ['429', '161', '5.0', '828124615'],
 ['429', '165', '4.0', '828124615'],
 ['429', '218', '4.0', '828124615']]

In [4]:
test[:5]

[['475', '3753', '4.5', '1498029450'],
 ['475', '122922', '4.5', '1498029469'],
 ['475', '6539', '4.5', '1498029484'],
 ['475', '152081', '4.5', '1498029487'],
 ['475', '122920', '3.5', '1498029495']]

In [5]:
user_dict = {}
item_dict = {}
user_count = 0
item_count = 0

for row in train:
    if row[0] not in user_dict:
        user_dict[row[0]] = user_count
        user_count += 1
    if row[1] not in item_dict:
        item_dict[row[1]] = item_count
        item_count += 1
        
for row in test:
    if row[0] not in user_dict:
        user_dict[row[0]] = user_count
        user_count += 1
    if row[1] not in item_dict:
        item_dict[row[1]] = item_count
        item_count += 1

In [6]:
user_dict_2 = {}
item_dict_2 = {}

for user in user_dict:
    user_dict_2[user_dict[user]] = user
for item in item_dict:
    item_dict_2[item_dict[item]] = item

In [7]:
for row in train:
    row[0] = user_dict[row[0]]
    row[1] = item_dict[row[1]]
    row[2] = float(row[2])
    
for row in test:
    row[0] = user_dict[row[0]]
    row[1] = item_dict[row[1]]
    row[2] = float(row[2])

In [8]:
train[:5]

[[0, 0, 4.0, '828124615'],
 [0, 1, 5.0, '828124615'],
 [0, 2, 5.0, '828124615'],
 [0, 3, 4.0, '828124615'],
 [0, 4, 4.0, '828124615']]

In [9]:
test[:5]

[[562, 2086, 4.5, '1498029450'],
 [562, 8255, 4.5, '1498029469'],
 [562, 3669, 4.5, '1498029484'],
 [562, 7933, 4.5, '1498029487'],
 [562, 7953, 3.5, '1498029495']]

### data size

In [10]:
n_user = len(user_dict)
n_item = len(item_dict)
n_rating = len(train)

print("number of ratings:", n_rating)
print("n_user:", n_user)
print("n_item:", n_item)

number of ratings: 90752
n_user: 610
n_item: 9724


### number of training and updating

In [11]:
n_train = len(train)
n_test = len(test)

print("n_train:", n_train)
print("n_test:", n_test)

n_train: 90752
n_test: 10084


### construct rating matrix

In [12]:
ratings = []
zeros = np.zeros(n_item)

for _ in range(n_user):
    ratings.append(zeros.copy())
    
for i in range(n_train):
    ratings[train[i][0]][train[i][1]] = 1.0

In [13]:
ratings[0][:10]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

### list rated movies for each user & for each movie

In [14]:
rate_lists_user = []
rate_lists_item = []

for i in range(n_user):
    tmp = []
    for j in range(n_item):
        if ratings[i][j] > 0.1:
            tmp.append(j)
    rate_lists_user.append(tmp.copy())
    
for i in range(n_item):
    tmp = []
    for j in range(n_user):
        if ratings[j][i] > 0.1:
            tmp.append(j)
    rate_lists_item.append(tmp.copy())

In [15]:
len(rate_lists_user[0])

58

In [16]:
len(rate_lists_item[0])

35

### initialize user and item vectors

In [17]:
user_vecs = []
item_vecs = []
dim = 100

def noise(dim):
    return np.random.uniform(-1, 1, dim)

for i in range(n_user):
    user_vecs.append(noise(dim))
for i in range(n_item):
    item_vecs.append(noise(dim))

In [18]:
user_vecs[0]

array([ 0.13447791, -0.99772485,  0.54392226, -0.09865682,  0.3268387 ,
       -0.86450701,  0.86861383, -0.79437957,  0.63928153,  0.12821373,
        0.57078482, -0.20018401, -0.13965783, -0.15969388,  0.5878453 ,
        0.58334237, -0.35916837, -0.21827213,  0.39158196,  0.97928394,
       -0.72248258, -0.78501067,  0.09447202, -0.15623347, -0.81354717,
       -0.46872609, -0.3706838 , -0.43456245,  0.15383748,  0.2583743 ,
        0.01727049,  0.25717319,  0.82749687,  0.37412997, -0.13522874,
        0.71559677,  0.18744917,  0.76963708,  0.65530759,  0.91469751,
       -0.9402911 ,  0.92568141,  0.0178753 ,  0.06657747,  0.70503105,
        0.70884874,  0.87074148, -0.22568737, -0.88382723,  0.61668535,
       -0.75293093, -0.07930588, -0.56211367, -0.39716758,  0.64389542,
        0.31833762,  0.04389545, -0.98658905,  0.15058619,  0.01592984,
        0.94582735,  0.61570376, -0.29114145,  0.09942552, -0.98652684,
       -0.33121659,  0.1130772 ,  0.48769633, -0.57065451, -0.74

### calculate $S^{user}$ and $S^{item}$

In [19]:
def vec_mul(arr):
    l = len(arr)
    ret = []
    
    for i in range(l):
        tmp = np.zeros(l)
        for j in range(l):
            tmp[j] = arr[i] * arr[j]
        ret.append(tmp)
        
    return ret

In [20]:
def update_s_user():
    for i in range(n_user):
        s = vec_mul(user_vecs[i])
        for j in range(dim):
            s_user[j] += s[j]

def update_s_item():
    for i in range(n_item):
        s = vec_mul(item_vecs[i])
        for j in range(dim):
            s_item[j] += s[j]

### gradient of loss

In [21]:
def dot(a, b):
    ret = 0.0
    for i in range(len(a)):
        ret += a[i] * b[i]
    return ret

def sign(x):
    if x >= 0:
        return 1
    else:
        return -1

def gradient_user(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * w_i * S^h
    for j in range(dim):
        ret[j] += 2 * alpha * dot(user_vecs[i], s_item[j])
    # \lambda * sign(w_i)
    for j in range(dim):
        ret[j] += lmd * sign(user_vecs[i][j])
    # -2 * \sum{h_j}
    for j in range(len(rate_lists_user[i])):
        no = rate_lists_user[i][j]
        rate = ratings[i][no]
        scalar = rate - (1-alpha) * dot(user_vecs[i], item_vecs[no])
        scalar *= -2
        ret += scalar * item_vecs[no]
    return ret

def gradient_item(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * h_i * S^w
    for j in range(dim):
        ret[j] += 2 * alpha * dot(item_vecs[i], s_user[j])
    # \lambda * sign(h_i)
    for j in range(dim):
        ret[j] += lmd * sign(item_vecs[i][j])
    # -2 * \sum{w_j}
    for j in range(len(rate_lists_item[i])):
        no = rate_lists_item[i][j]
        rate = ratings[no][i]
        scalar = rate - (1-alpha) * dot(item_vecs[i], user_vecs[no])
        scalar *= -2
        ret += scalar * user_vecs[no]
    return ret

def gradient_user_u(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * w_i * S^h
    for j in range(dim):
        ret[j] += 2 * alpha * dot(user_vecs_u[i], s_item_u[j])
    # \lambda * sign(w_i)
    for j in range(dim):
        ret[j] += lmd * sign(user_vecs_u[i][j])
    # -2 * \sum{h_j}
    for j in range(len(rate_lists_user_u[i])):
        no = rate_lists_user_u[i][j]
        rate = ratings_u[i][no]
        scalar = rate - (1-alpha) * dot(user_vecs_u[i], item_vecs_u[no])
        scalar *= -2
        ret += scalar * item_vecs_u[no]
    return ret

def gradient_item_u(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * h_i * S^w
    for j in range(dim):
        ret[j] += 2 * alpha * dot(item_vecs_u[i], s_user_u[j])
    # \lambda * sign(h_i)
    for j in range(dim):
        ret[j] += lmd * sign(item_vecs_u[i][j])
    # -2 * \sum{w_j}
    for j in range(len(rate_lists_item_u[i])):
        no = rate_lists_item_u[i][j]
        rate = ratings_u[no][i]
        scalar = rate - (1-alpha) * dot(item_vecs_u[i], user_vecs_u[no])
        scalar *= -2
        ret += scalar * user_vecs_u[no]
    return ret

### hyperparameters

In [22]:
lmd = 0.01
alpha = 0.0312
rho = alpha * (n_user*n_item - n_train) / n_train
EPOCH = 60
lr = 0.001 # the paper use line search, but setting learning rate is good enough

print("lambda:", lmd)
print("alpha:", alpha)
print("rho:", rho)
print("learning rate:", lr)

lambda: 0.01
alpha: 0.0312
rho: 2.00806269393512
learning rate: 0.001


### train

In [23]:
def update_user(i, scalar):
    mat = vec_mul(user_vecs[i])
    for j in range(dim):
        s_user[j] += scalar * mat[j]
    return

def update_item(i, scalar):
    mat = vec_mul(item_vecs[i])
    for j in range(dim):
        s_item[j] += scalar * mat[j]
    return

def update_user_u(i, scalar):
    mat = vec_mul(user_vecs_u[i])
    for j in range(dim):
        s_user_u[j] += scalar * mat[j]
    return

def update_item_u(i, scalar):
    mat = vec_mul(item_vecs_u[i])
    for j in range(dim):
        s_item_u[j] += scalar * mat[j]
    return

In [24]:
# def cal_loss():
#     ret = 0.0
#     for i in range(n_user):
#         for j in range(n_item):
#             if ratings[i][j] > 0.1:
#                 residual = ratings[i][j] - dot(user_vecs[i], item_vecs[j])
#                 ret += residual ** 2
#             else:
#                 residual = ratings[i][j] - dot(user_vecs[i], item_vecs[j])
#                 ret += alpha * (residual ** 2)
#     sum_w = 0.0
#     for i in range(n_user):
#         for j in range(dim):
#             sum_w += sign(user_vecs[i][j]) * user_vecs[i][j]
#     for i in range(n_item):
#         for j in range(dim):
#             sum_w += sign(item_vecs[i][j]) * item_vecs[i][j]
#     return ret + lmd * sum_w

# def cal_loss_u():
#     ret = 0.0
#     for i in range(n_user):
#         for j in range(n_item):
#             if ratings_u[i][j] > 0.1:
#                 residual = ratings_u[i][j] - dot(user_vecs_u[i], item_vecs_u[j])
#                 ret += residual ** 2
#             else:
#                 residual = ratings_u[i][j] - dot(user_vecs_u[i], item_vecs_u[j])
#                 ret += alpha * (residual ** 2)
#     sum_w = 0.0
#     for i in range(n_user):
#         for j in range(dim):
#             sum_w += sign(user_vecs_u[i][j]) * user_vecs_u[i][j]
#     for i in range(n_item):
#         for j in range(dim):
#             sum_w += sign(item_vecs_u[i][j]) * item_vecs_u[i][j]
#     return ret + lmd * sum_w

In [25]:
def cal_acc(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs[test[i][0]], item_vecs[test[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])

    return error / n

def cal_acc_u(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs_u[test[i][0]], item_vecs_u[test[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])

    return error / n

def cal_insample_acc(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs[train[i][0]], item_vecs[train[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])

    return error / n

def cal_insample_acc_u(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs_u[train[i][0]], item_vecs_u[train[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])

    return error / n

In [26]:
user_vecs = []
item_vecs = []
s_user = vec_mul(np.zeros(dim))
s_item = vec_mul(np.zeros(dim))

for i in range(n_user):
    user_vecs.append(noise(dim))
for i in range(n_item):
    item_vecs.append(noise(dim))

update_s_user()
update_s_item()

user_vecs_u = copy.deepcopy(user_vecs)
item_vecs_u = copy.deepcopy(item_vecs)
s_user_u = copy.deepcopy(s_user)
s_item_u = copy.deepcopy(s_item)

In [27]:
user_vecs = copy.deepcopy(user_vecs_u)
item_vecs = copy.deepcopy(item_vecs_u)
s_user = copy.deepcopy(s_user_u)
s_item = copy.deepcopy(s_item_u)

start_train_time = time.time()
cal_loss_time = 0.0
cal_acc_time = 0.0

losses = []
in_accs = []
out_accs = []

In [28]:
for epoch in range(EPOCH):
    lis_user = list(range(n_user))
    lis_item = list(range(n_item))
    random.shuffle(lis_user)
    random.shuffle(lis_item)
    count = 0
    
    for i in lis_user:
        grad = gradient_user(i, alpha, lmd)
        update_user(i, -1)
        user_vecs[i] -= lr * grad
        update_user(i, 1)
        
        if count % 100 == 99:
            print("user:", count+1, "; in-sample error:", cal_insample_acc(n_test), "; out-sample error", cal_acc(n_test))
        count += 1
    
    count = 0
    
    for i in lis_item:
        grad = gradient_item(i, alpha, lmd)
        update_item(i, -1)
        item_vecs[i] -= lr * grad
        update_item(i, 1)
        
        if count % 100 == 99:
            print("item:", count+1, "; in-sample error:", cal_insample_acc(n_test), "; out-sample error", cal_acc(n_test))
        count += 1
    
    loss_time = time.time()
    # loss = 0
    # loss = cal_loss()
    # losses.append(loss)
    cal_loss_time += time.time() - loss_time
    
    acc_time = time.time()
    in_acc = cal_insample_acc(n_test)
    out_acc = cal_acc(n_test)
    in_accs.append(in_acc)
    out_accs.append(out_acc)
    cal_acc_time += time.time() - acc_time
    
    print("\nepoch", epoch, ": averge error(in):", in_acc, "; averge error(out):", out_acc, "\n")

print("total training time:", time.time()-start_train_time)
print("cal loss time:", cal_loss_time)
print("cal accuracy time:", cal_acc_time)

user: 100 ; in-sample error: 2.65129066616 ; out-sample error 2.63783889084
user: 200 ; in-sample error: 2.50091017658 ; out-sample error 2.59931618369
user: 300 ; in-sample error: 2.31622411336 ; out-sample error 2.57183594563
user: 400 ; in-sample error: 2.18865226181 ; out-sample error 2.31441948584
user: 500 ; in-sample error: 2.04115455508 ; out-sample error 2.22964259791
user: 600 ; in-sample error: 1.91916451362 ; out-sample error 2.17476805112
item: 100 ; in-sample error: 1.89482529045 ; out-sample error 2.17133565992
item: 200 ; in-sample error: 1.89450013268 ; out-sample error 2.17122257382
item: 300 ; in-sample error: 1.8929861583 ; out-sample error 2.17060093292
item: 400 ; in-sample error: 1.89238365953 ; out-sample error 2.17031432359
item: 500 ; in-sample error: 1.89226791384 ; out-sample error 2.1701958682
item: 600 ; in-sample error: 1.89091022628 ; out-sample error 2.16990979981
item: 700 ; in-sample error: 1.89061242878 ; out-sample error 2.16932484422
item: 800 ; in

user: 400 ; in-sample error: 1.51278423627 ; out-sample error 1.85589380035
user: 500 ; in-sample error: 1.43314125718 ; out-sample error 1.82227838482
user: 600 ; in-sample error: 1.35956252467 ; out-sample error 1.7817288621
item: 100 ; in-sample error: 1.35024221499 ; out-sample error 1.77858878298
item: 200 ; in-sample error: 1.34984420584 ; out-sample error 1.77852546681
item: 300 ; in-sample error: 1.3494482857 ; out-sample error 1.77837443097
item: 400 ; in-sample error: 1.3488739664 ; out-sample error 1.77828583489
item: 500 ; in-sample error: 1.34811400868 ; out-sample error 1.77821809792
item: 600 ; in-sample error: 1.3478443162 ; out-sample error 1.77815415988
item: 700 ; in-sample error: 1.34753252859 ; out-sample error 1.77803768199
item: 800 ; in-sample error: 1.34730161436 ; out-sample error 1.77793786498
item: 900 ; in-sample error: 1.3465222459 ; out-sample error 1.77779459969
item: 1000 ; in-sample error: 1.3464014216 ; out-sample error 1.7775371088
item: 1100 ; in-sa

item: 100 ; in-sample error: 1.07096116067 ; out-sample error 1.50797537816
item: 200 ; in-sample error: 1.07067587822 ; out-sample error 1.50795665126
item: 300 ; in-sample error: 1.07062842718 ; out-sample error 1.50791855319
item: 400 ; in-sample error: 1.07044493765 ; out-sample error 1.50788142396
item: 500 ; in-sample error: 1.07003975208 ; out-sample error 1.50781441555
item: 600 ; in-sample error: 1.06996803188 ; out-sample error 1.50778478482
item: 700 ; in-sample error: 1.06957576679 ; out-sample error 1.50764891596
item: 800 ; in-sample error: 1.06924667313 ; out-sample error 1.50757161658
item: 900 ; in-sample error: 1.0691542609 ; out-sample error 1.50756042838
item: 1000 ; in-sample error: 1.06906258995 ; out-sample error 1.50753311296
item: 1100 ; in-sample error: 1.06889314519 ; out-sample error 1.50751344348
item: 1200 ; in-sample error: 1.0686168093 ; out-sample error 1.50743475631
item: 1300 ; in-sample error: 1.0682027607 ; out-sample error 1.50739040406
item: 1400 

item: 400 ; in-sample error: 0.921331703662 ; out-sample error 1.31941329936
item: 500 ; in-sample error: 0.921211579931 ; out-sample error 1.31940043052
item: 600 ; in-sample error: 0.921083958337 ; out-sample error 1.31938017901
item: 700 ; in-sample error: 0.920938007663 ; out-sample error 1.31936011859
item: 800 ; in-sample error: 0.920890606413 ; out-sample error 1.31934470538
item: 900 ; in-sample error: 0.920843633727 ; out-sample error 1.31931462724
item: 1000 ; in-sample error: 0.920702296501 ; out-sample error 1.31930429818
item: 1100 ; in-sample error: 0.920543874725 ; out-sample error 1.31928684597
item: 1200 ; in-sample error: 0.920402516558 ; out-sample error 1.31928261102
item: 1300 ; in-sample error: 0.920276705421 ; out-sample error 1.3192731498
item: 1400 ; in-sample error: 0.920047125893 ; out-sample error 1.31925940508
item: 1500 ; in-sample error: 0.919988734292 ; out-sample error 1.31924672579
item: 1600 ; in-sample error: 0.919889911994 ; out-sample error 1.31922

item: 600 ; in-sample error: 0.843254049288 ; out-sample error 1.18806221729
item: 700 ; in-sample error: 0.843141308798 ; out-sample error 1.18805686938
item: 800 ; in-sample error: 0.843135732888 ; out-sample error 1.18805000485
item: 900 ; in-sample error: 0.843116022276 ; out-sample error 1.18805265983
item: 1000 ; in-sample error: 0.842918937501 ; out-sample error 1.1880402206
item: 1100 ; in-sample error: 0.842828791303 ; out-sample error 1.18802104703
item: 1200 ; in-sample error: 0.842731253947 ; out-sample error 1.18801750799
item: 1300 ; in-sample error: 0.842613700537 ; out-sample error 1.18800963655
item: 1400 ; in-sample error: 0.842600138566 ; out-sample error 1.18800443673
item: 1500 ; in-sample error: 0.842583542819 ; out-sample error 1.18800140883
item: 1600 ; in-sample error: 0.842566143684 ; out-sample error 1.18799189448
item: 1700 ; in-sample error: 0.842390711173 ; out-sample error 1.18798602908
item: 1800 ; in-sample error: 0.842330818915 ; out-sample error 1.187

...

item: 1400 ; in-sample error: 0.757548680993 ; out-sample error 1.01502445342
item: 1500 ; in-sample error: 0.757512403896 ; out-sample error 1.01502199436
item: 1600 ; in-sample error: 0.757499773355 ; out-sample error 1.01501840241
item: 1700 ; in-sample error: 0.757379326586 ; out-sample error 1.0150146183
item: 1800 ; in-sample error: 0.75731326226 ; out-sample error 1.01501256069
item: 1900 ; in-sample error: 0.757265895931 ; out-sample error 1.01500877213
item: 2000 ; in-sample error: 0.757161710569 ; out-sample error 1.01500562474
item: 2100 ; in-sample error: 0.757140529034 ; out-sample error 1.01500628381
item: 2200 ; in-sample error: 0.75713695584 ; out-sample error 1.01500623599
item: 2300 ; in-sample error: 0.757065834466 ; out-sample error 1.01500372963
item: 2400 ; in-sample error: 0.756979834464 ; out-sample error 1.01499942788
item: 2500 ; in-sample error: 0.756937623879 ; out-sample error 1.01499763764
item: 2600 ; in-sample error: 0.756804035582 ; out-sample error 1.0

item: 1600 ; in-sample error: 0.745166375319 ; out-sample error 1.01174327371
item: 1700 ; in-sample error: 0.745138655436 ; out-sample error 1.01174252786
item: 1800 ; in-sample error: 0.745133882175 ; out-sample error 1.01173754212
item: 1900 ; in-sample error: 0.745099160121 ; out-sample error 1.01173865667
item: 2000 ; in-sample error: 0.745058432156 ; out-sample error 1.01173207493
item: 2100 ; in-sample error: 0.74502469618 ; out-sample error 1.01172643339
item: 2200 ; in-sample error: 0.744782017701 ; out-sample error 1.0117196178
item: 2300 ; in-sample error: 0.744770389816 ; out-sample error 1.01171894306
item: 2400 ; in-sample error: 0.74466798596 ; out-sample error 1.01171610159
item: 2500 ; in-sample error: 0.744493424488 ; out-sample error 1.01170747219
item: 2600 ; in-sample error: 0.744457310082 ; out-sample error 1.01170452171
item: 2700 ; in-sample error: 0.744447487106 ; out-sample error 1.01170265765
item: 2800 ; in-sample error: 0.744338879814 ; out-sample error 1.0

item: 1800 ; in-sample error: 0.732779902092 ; out-sample error 1.00888511085
item: 1900 ; in-sample error: 0.732603705951 ; out-sample error 1.00887886454
item: 2000 ; in-sample error: 0.732544223913 ; out-sample error 1.00887614077
item: 2100 ; in-sample error: 0.732413666111 ; out-sample error 1.00887797779
item: 2200 ; in-sample error: 0.732323872126 ; out-sample error 1.00887818632
item: 2300 ; in-sample error: 0.732296734934 ; out-sample error 1.00887128903
item: 2400 ; in-sample error: 0.73224998712 ; out-sample error 1.00886156583
item: 2500 ; in-sample error: 0.732231223984 ; out-sample error 1.00886182533
item: 2600 ; in-sample error: 0.732179333691 ; out-sample error 1.00885862607
item: 2700 ; in-sample error: 0.732130749175 ; out-sample error 1.0088553723
item: 2800 ; in-sample error: 0.731990136246 ; out-sample error 1.00885098962
item: 2900 ; in-sample error: 0.731779890487 ; out-sample error 1.00884525891
item: 3000 ; in-sample error: 0.731724008083 ; out-sample error 1.

item: 2000 ; in-sample error: 0.71889519928 ; out-sample error 1.00670953876
item: 2100 ; in-sample error: 0.718745347436 ; out-sample error 1.00670869192
item: 2200 ; in-sample error: 0.718715687941 ; out-sample error 1.00670549227
item: 2300 ; in-sample error: 0.718662910462 ; out-sample error 1.00670152128
item: 2400 ; in-sample error: 0.718623344347 ; out-sample error 1.00669392138
item: 2500 ; in-sample error: 0.718608511974 ; out-sample error 1.00669194934
item: 2600 ; in-sample error: 0.718578854573 ; out-sample error 1.00668745195
item: 2700 ; in-sample error: 0.718544922783 ; out-sample error 1.00668310427
item: 2800 ; in-sample error: 0.718469689071 ; out-sample error 1.00667780679
item: 2900 ; in-sample error: 0.718428139609 ; out-sample error 1.00667361472
item: 3000 ; in-sample error: 0.718318989937 ; out-sample error 1.00666852897
item: 3100 ; in-sample error: 0.718235509858 ; out-sample error 1.0066662606
item: 3200 ; in-sample error: 0.7181286268 ; out-sample error 1.00

KeyboardInterrupt: 

### update & predict

In [29]:
user_vecs_u = copy.deepcopy(user_vecs)
item_vecs_u = copy.deepcopy(item_vecs)
s_user_u = copy.deepcopy(s_user)
s_item_u = copy.deepcopy(s_item)
ratings_u = copy.deepcopy(ratings)
rate_lists_user_u = copy.deepcopy(rate_lists_user)
rate_lists_item_u = copy.deepcopy(rate_lists_item)

In [30]:
not_rating_list = []

for i in range(n_user):
    tmp = []
    for j in range(n_item):
        if j not in rate_lists_user_u:
            tmp.append(j)
    not_rating_list.append(tmp.copy())

In [ ]:
def take_value(elem):
    return elem[1]

update_epoch = 500
update_lr = 0.005
update_time_for_one_print = 0.0
predicts = []
    
for i in range(n_test):
    # predict
    tmp = []
    for j in not_rating_list[test[i][0]]:
        tmp.append([item_dict_2[j], dot(user_vecs_u[test[i][0]], item_vecs_u[j])])
    tmp.sort(key = take_value)

    row = []
    row.append(user_dict_2[test[i][0]]) # user_id
    row.append(test[i][3]) # time

    for j in range(100):
        row.append(tmp[j][0])

    predicts.append(row)

    # update model
    ratings_u[test[i][0]][test[i][1]] = 1.0

    if test[i][1] not in rate_lists_user_u:
        rate_lists_user_u.append(test[i][1])
        not_rating_list[test[i][0]].remove(test[i][1])
    if test[i][0] not in rate_lists_item_u:
        rate_lists_item_u.append(test[i][0])
        
    start_update_time = time.time()

    for epoch in range(update_epoch):
        # update user vector
        grad = gradient_user_u(test[i][0], alpha, lmd)
        update_user_u(test[i][0], -1)
        user_vecs_u[test[i][0]] -= update_lr * grad
        update_user_u(test[i][0], 1)
        # update item vector
        grad = gradient_item_u(test[i][1], alpha, lmd)
        update_item_u(test[i][1], -1)
        item_vecs_u[test[i][1]] -= update_lr * grad
        update_item_u(test[i][1], 1)
        
    update_time_for_one_print += time.time() - start_update_time

    if i % 10 == 9:
        in_acc = cal_insample_acc_u(n_test)
        out_acc = cal_acc_u(n_test)
        in_accs.append(in_acc)
        out_accs.append(out_acc)
        print(i+1, ": averge error(in):", cal_insample_acc_u(1000), "; averge error(out):", cal_acc_u(1000))
        print("time consumed:", update_time_for_one_print)
        update_time_for_one_print = 0.0

In [ ]:
with open("perdict_online.csv", "w", newline = "") as f:
    w = csv.writer(f)
    for row in predicts:
        w.writerow(row)

### test performance (in-sample after update)

In [ ]:
predict = []
rates = []

# for i in range(n_train):
#     rates.append(1.0)
#     predict.append(dot(user_vecs_u[train[i][0]], item_vecs_u[train[i][1]]))
    
for i in range(n_test):
    rates.append(1.0)
    predict.append(dot(user_vecs_u[test[i][0]], item_vecs_u[test[i][1]]))

In [ ]:
for i in range(30):
    print("ans:", rates[i], "; predict:", predict[i])

In [ ]:
error = 0.0

for i in range(len(rates)):
    error += abs(rates[i] - predict[i])**2
    
print("averge error:", error / len(rates))

### plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.plot(losses)
# plt.xlabel("epoch")
# plt.ylabel("loss")
# plt.show()

In [ ]:
plt.plot(in_accs)
plt.plot(out_accs)
plt.xlabel("epoch")
plt.ylabel("error")
plt.show()